# 1a - Setup - Stores variables that describe the environment
This notebook assumes the chatbot-demo-lambda-opensearch-setup which is part of the chatbot-demo CloudFormation stack has been run.  It stores variables needed by subsequent workbooks. If the chatbot demo CloudFormation stack was run with parameter CreateLambda set to No then notebook 20b should be run instead.

In [ ]:
!pip install opensearch-py opensearch-py-ml accelerate sentence-transformers deprecated pandas==2.0.3

In [ ]:
import os
import boto3
import json
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from opensearch_py_ml.ml_commons import MLCommonClient

#### Get a boto3 session and retrieve the region name

In [ ]:
session = boto3.session.Session()
region_name = session.region_name

print("Region is", region_name)
%store region_name

#### Set the name of the OpenSearch domain created by the CloudFormation stack.  Should be chatbot-demo.

In [ ]:
opensearch_domain_name = "chatbot-demo"

#### Retrieve the endpoint of the OpenSearch domain created by the CloudFormation stack.

In [ ]:
opensearch_boto3_client = boto3.client("opensearch")
response = opensearch_boto3_client.describe_domain(
    DomainName = opensearch_domain_name
)
host = response['DomainStatus']['Endpoints']['vpc']
%store host
print("The endpoint for the OpenSearch domain", opensearch_domain_name, "is", host)

#### Get OpenSearch client

In [ ]:
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region_name)

opensearch_client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

#### Get OpenSearch model ID

In [ ]:
ml_client = MLCommonClient(opensearch_client)

model_query = {
  "query": {
    "match_all": {}
  },
  "size": 1000
}

model_query_json = json.dumps(model_query)
response = ml_client.search_model(model_query)
model_id = response['hits']['hits'][0]['_source']['model_id']

#### Store key parameters for later use

In [ ]:
summary_index_name = "chatbot-summary"
full_text_index_name = "chatbot-full_text"
pipeline_id = "chatbot-nlp-pipeline"
%store summary_index_name
%store full_text_index_name
%store pipeline_id
%store host
%store model_id

#### Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#### SPDX-License-Identifier: MIT-0